# 电子邮件协议

我们先来看一下一封电子邮件从发送到接收的全部过程：

![Email Flow](../images/email_flow.png)

1. 发件人使用邮箱客户端(MUA, Mail User Agent)来编写电子邮件，然后点击发送；
2. MUA并不是把邮件直接发送给收件人，而是发送给了邮件传输代理(MTA，Mail Transfer Agent)，这个传输代理一般是发送人的邮箱提供商，比如163.com
3. 发送方的MTA会把邮件发送给接收人邮箱的MTA，比如qq.com，这中间可能还会经过其他的MTA。
4. 收件人的MTA收到邮件后，会把邮件投递到邮件投递代理（MDA：Mail Delivery Agent），Email到达MDA后，就静静地躺在接收邮件代理商的某个服务器上，存放在某个文件或特殊的数据库里，我们将这个长期保存邮件的地方称之为电子邮箱。
5. 接收人必须通过一个MUA来从MDA中主动获取邮件。

其中第2到第4步都会使用SMTP（Simple Mail Transfer Protocol）协议，而第5步则可以使用POP3(Post Office Protocol Version 3)和IMAP 4(Internet Message Access Protocol Version 4)协议。

## 发送电子邮件

最全参考：https://www.jianshu.com/p/abb2d6e91c1f

最新的接口使用示例：https://docs.python.org/3/library/email.examples.html

Python对SMTP支持有`smtplib`和`email`两个模块，`email`负责构造邮件，`smtplib`负责发送邮件

In [1]:
import smtplib

## 构造一段普通的文件

In [2]:
# ASCII码的邮件内容，如果只包括ascii码，则可以直接发送
message_ascii = """From: Yansheng <200822105@qq.com>
To: Yansheng <yangyansheng@sensetime.com>
Subject: SMTP eamil test

This is a test e-mail message
"""

## 带中文的邮件消息

In [3]:
from email.mime.text import MIMEText
from email.header import Header
from email.utils import parseaddr, formataddr

def _format_addr(s):
    # 杨延生 <200822105@qq.com>　-> 杨延生　200822105@qq.com
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

msg_conent = '你好，这是一封测试邮件'
msg_zh = MIMEText(msg_conent, 'plain', 'utf-8')
msg_zh['Subject'] = Header('SMTP测试邮件', 'utf-8')
msg_zh['From'] = _format_addr('杨延生 <200822105@qq.com>')
msg_zh['To'] = _format_addr('杨延生 <yangyansheng@sensetime.com>')
print(msg_zh.as_string())

Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
Subject: =?utf-8?b?U01UUOa1i+ivlemCruS7tg==?=
From: =?utf-8?b?5p2o5bu255Sf?= <200822105@qq.com>
To: =?utf-8?b?5p2o5bu255Sf?= <yangyansheng@sensetime.com>

5L2g5aW977yM6L+Z5piv5LiA5bCB5rWL6K+V6YKu5Lu2



## HTML格式的邮件

In [4]:
msg_conent = "你好：<h1>这是一封测试邮件</h1><p><a href='http://www.baidu.com'>百度</a></p>"
msg_html = MIMEText(msg_conent, 'html', 'utf-8')
msg_html['Subject'] = Header('SMTP测试邮件', 'utf-8')
msg_html['From'] = _format_addr('杨延生 <200822105@qq.com>')
msg_html['To'] = _format_addr('杨延生 <yangyansheng@sensetime.com>')
print(msg_html.as_string())

Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
Subject: =?utf-8?b?U01UUOa1i+ivlemCruS7tg==?=
From: =?utf-8?b?5p2o5bu255Sf?= <200822105@qq.com>
To: =?utf-8?b?5p2o5bu255Sf?= <yangyansheng@sensetime.com>

5L2g5aW977yaPGgxPui/meaYr+S4gOWwgea1i+ivlemCruS7tjwvaDE+PHA+PGEgaHJlZj0naHR0
cDovL3d3dy5iYWlkdS5jb20nPueZvuW6pjwvYT48L3A+



## 带附件的邮件

In [7]:
from email.mime.multipart import MIMEMultipart

msg_attachment = MIMEMultipart('related')
msg_attachment['Subject'] = Header('SMTP测试邮件', 'utf-8')
msg_attachment['From'] = _format_addr('杨延生 <200822105@qq.com>')
msg_attachment['To'] = _format_addr('杨延生 <yangyansheng@sensetime.com>')

# 添加一个文本附件
from email.mime.application import MIMEApplication
txt_attachment = MIMEApplication(open('email.ipynb').read())
txt_attachment.add_header('Content-Disposition', 'attachment', filename='email.ipynb')
msg_attachment.attach(txt_attachment)

# 添加一个图片附件
from email.mime.image import MIMEImage
img_attachment = MIMEImage(open('../images/email_flow.png', 'rb').read())
img_attachment.add_header('Content-ID', '<image>')
msg_attachment.attach(img_attachment)

# 添加带图片url的html正文
msg_img_html = """
你好：<p>这是一封测试邮件</p><p><a href='http://www.baidu.com'>百度</a></p>
<p>![](cid:image)</p>
"""
msg_attachment.attach(MIMEText(msg_img_html, 'html', 'utf-8'))

## 发送邮件

In [ ]:
# 发送方邮件账户
sender = '200822105@qq.com'
# 什么是授权码？　https://service.mail.qq.com/cgi-bin/help?subtype=1&&no=1001256&&id=28
password = 'vyrwzmwyasfycaja' # QQ邮箱16位授权码
# 输入SMTP服务器地址
smtp_server = 'smtp.qq.com'

# 接收方，可以有多个地址
reciver = ['yangyansheng@sensetime.com']

server = smtplib.SMTP_SSL(smtp_server)
server.set_debuglevel(1)
server.login(sender, password)
server.sendmail(sender, reciver, msg_attachment.as_string())
server.quit()

## 接收邮件

收取邮件就是编写一个MUA作为客户端，从MDA把邮件获取到用户的电脑或者手机上。收取邮件最常用的协议是POP协议，目前版本号是3，俗称POP3。Python内置一个poplib模块，实现了POP3协议，可以直接用来收邮件。

注意到POP3协议收取的不是一个已经可以阅读的邮件本身，而是邮件的原始文本，这和SMTP协议很像，SMTP发送的也是经过编码后的一大段文本。要把POP3收取的文本变成可以阅读的邮件，还需要用email模块提供的各种类来解析原始文本，变成可阅读的邮件对象。

所以，收取邮件分两步：
- 第一步：用poplib把邮件的原始文本下载到本地；
- 第二部：用email解析原始文本，还原为邮件对象。

In [ ]:
import poplib

email = '200822105@qq.com'
password = 'vyrwzmwyasfycaja'
pop3_server = 'pop.qq.com'

server = poplib.POP3(pop3_server)
server.set_debuglevel(1)
print(server.getwelcome().decode('utf-8'))

server.user(email)
server.pass_(password)

print('Mails: %s. Size: %s' % server.stat())

resp, mails, octets = server.list()
print(mails)

index = len(mails)
resp,lines,octets = server.retr(index)

msg_content = b'\r\n'.join(lines).decode('utf-8')

print(msg_content)

from email.parser import Parser

msg = Parser().parsestr(msg_content)

In [13]:
from email.header import decode_header
from email.utils import parseaddr

# indent用于缩进显示:
def print_info(msg, indent=0):
    if indent == 0:
        for header in ['From', 'To', 'Subject']:
            value = msg.get(header, '')
            if value:
                if header=='Subject':
                    value = decode_str(value)
                else:
                    hdr, addr = parseaddr(value)
                    name = decode_str(hdr)
                    value = u'%s <%s>' % (name, addr)
            print('%s%s: %s' % ('  ' * indent, header, value))
    if (msg.is_multipart()):
        parts = msg.get_payload()
        for n, part in enumerate(parts):
            print('%spart %s' % ('  ' * indent, n))
            print('%s--------------------' % ('  ' * indent))
            print_info(part, indent + 1)
    else:
        content_type = msg.get_content_type()
        if content_type=='text/plain' or content_type=='text/html':
            content = msg.get_payload(decode=True)
            charset = guess_charset(msg)
            if charset:
                content = content.decode(charset)
            print('%sText: %s' % ('  ' * indent, content + '...'))
        else:
            print('%sAttachment: %s' % ('  ' * indent, content_type))
        
def decode_str(s):
    value, charset = decode_header(s)[0]
    if charset:
        value = value.decode(charset)
    return value

def guess_charset(msg):
    charset = msg.get_charset()
    if charset is None:
        content_type = msg.get('Content-Type', '').lower()
        pos = content_type.find('charset=')
        if pos >= 0:
            charset = content_type[pos + 8:].strip()
    return charset

In [ ]:
print_info(msg)

![Images](../images/qqmail-inbox.png)